In [7]:
from __future__ import absolute_import
from __future__ import print_function

import random
from keras.models import Model
from keras.layers import Input, Flatten, Dense, Dropout, Lambda
# from keras.optimizers import RMSprop
from keras import backend as K
import keras 
from keras.datasets import mnist 
from keras.models import Sequential 
from keras.layers import Dense, Dropout 
from keras.optimizers import RMSprop, Adam, SGD, Adadelta 
import numpy as np 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
# print(os.listdir("../input"))
import matplotlib.pyplot as plt
get_ipython().magic(u'matplotlib inline')
plt.style.use('ggplot')

In [17]:
num_classes = 10

In [19]:
def euclidean_distance(vects):
    x, y = vects
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))


def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)


In [20]:

def contrastive_loss(y_true, y_pred):
    margin = 1
    sqaure_pred = K.square(y_pred)
    margin_square = K.square(K.maximum(margin - y_pred, 0))
    return K.mean(y_true * sqaure_pred + (1 - y_true) * margin_square)


In [33]:
def compute_accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    pred = y_pred.ravel() < 0.5
    return np.mean(pred == y_true)


def accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    return K.mean(K.equal(y_true, K.cast(y_pred < 0.5, y_true.dtype)))

In [21]:

def create_pairs(x, digit_indices):
    pairs = []
    labels = []
    n = min([len(digit_indices[d]) for d in range(num_classes)]) - 1
    for d in range(num_classes):
        for i in range(n):
            z1, z2 = digit_indices[d][i], digit_indices[d][i + 1]
            pairs += [[x[z1], x[z2]]]
            inc = random.randrange(1, num_classes)
            dn = (d + inc) % num_classes
            z1, z2 = digit_indices[d][i], digit_indices[dn][i]
            pairs += [[x[z1], x[z2]]]
            labels += [1, 0]
    return np.array(pairs), np.array(labels)



In [22]:
def create_model(input_shape):
    input = Input(shape=input_shape)
    x = Flatten()(input)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(128, activation='relu')(x)
    x = Lambda(lambda  x: K.l2_normalize(x,axis=1))(x)
    x = Lambda(lambda  x: K.l2_normalize(x,axis=1))(x)
    return Model(input, x)


In [12]:
(x_train, y_train), (x_test, y_test) = mnist.load_data() 
x_train.shape,x_test.shape

11493376/11490434 [==============================] - 0s 0us/step


((60000, 28, 28), (10000, 28, 28))

In [14]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
test_x = y_test/255
input_shape = x_train.shape[1:]

In [29]:
digit_indices = [np.where(y_train == i)[0] for i in range(10)]

tr_pairs, tr_y = create_pairs(x_train, digit_indices)
digit_indices = [np.where(y_test == i)[0] for i in range(num_classes)]
te_pairs, te_y = create_pairs(x_test, digit_indices)
te_pairs = te_pairs.astype('float32')
tr_pairs = tr_pairs.astype('float32')
te_y = te_y.astype('float32')
tr_y = tr_y.astype('float32')


In [30]:
base_model = create_model(input_shape)

input_a = Input(shape=input_shape)
input_b = Input(shape=input_shape)

# because we re-use the same instance `base_network`,
# the weights of the network
# will be shared across the two branches
processed_a = base_model(input_a)
processed_b = base_model(input_b)


In [31]:
distance = Lambda(euclidean_distance, output_shape=eucl_dist_output_shape)([processed_a, processed_b])

model = Model([input_a, input_b], distance)
opt = RMSprop()


In [34]:
model.compile(loss=contrastive_loss, optimizer=opt, metrics=['accuracy'])
# train
history = model.fit([tr_pairs[:, 0], tr_pairs[:, 1]], tr_y,
          batch_size=128,
          epochs=20,
          validation_data=([te_pairs[:, 0], te_pairs[:, 1]], te_y))

# compute final accuracy on training and test sets
y_pred_tr = model.predict([tr_pairs[:, 0], tr_pairs[:, 1]])
tr_acc = compute_accuracy(tr_y, y_pred_tr)
y_pred_te = model.predict([te_pairs[:, 0], te_pairs[:, 1]])
te_acc = compute_accuracy(te_y, y_pred_te)

print('* Accuracy on training set: %0.2f%%' % (100 * tr_acc))
print('* Accuracy on test set: %0.2f%%' % (100 * te_acc))

Epoch 1/20
847/847 [==============================] - 5s 4ms/step - loss: 0.0153 - accuracy: 0.0183 - val_loss: 0.0220 - val_accuracy: 0.0283
Epoch 2/20
847/847 [==============================] - 4s 4ms/step - loss: 0.0150 - accuracy: 0.0179 - val_loss: 0.0205 - val_accuracy: 0.0258
Epoch 3/20
847/847 [==============================] - 4s 4ms/step - loss: 0.0152 - accuracy: 0.0179 - val_loss: 0.0205 - val_accuracy: 0.0258
Epoch 4/20
847/847 [==============================] - 4s 4ms/step - loss: 0.0145 - accuracy: 0.0178 - val_loss: 0.0190 - val_accuracy: 0.0233
Epoch 5/20
847/847 [==============================] - 4s 4ms/step - loss: 0.0136 - accuracy: 0.0161 - val_loss: 0.0204 - val_accuracy: 0.0249
Epoch 6/20
847/847 [==============================] - 4s 4ms/step - loss: 0.0147 - accuracy: 0.0175 - val_loss: 0.0195 - val_accuracy: 0.0250
Epoch 7/20
847/847 [==============================] - 4s 4ms/step - loss: 0.0132 - accuracy: 0.0156 - val_loss: 0.0197 - val_accuracy: 0.0250
Epoch 

The main advantages of Siamese Networks are -




• More Robust to class Imbalance: With the aid of One-shot learning, given a few images per class
is sufficient for Siamese Networks to recognize those images in the future.

• Nice to an ensemble with the best classifier: Given that its learning mechanism is somewhat
different from Classification, simple averaging of it with a Classifier can do much better than
average 2 correlated Supervised models (e.g. GBM & RF classifier).

• Learning from Semantic Similarity: Siamese focuses on learning embeddings (in the deeper layer)
that place the same classes/concepts close together. Hence, can learn semantic similarity.

The downsides of the Siamese Networks can be,

• Needs more training time than normal networks: Since Siamese Networks involves quadratic pairs
to learn from (to see all information available) it is slower than normal classification type of learning(pointwise learning)

• Doesn’t output probabilities: Since training involves pairwise learning, it won’t output the probabilities of the prediction, but the distance from each class.